In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from os import environ
environ['DATA_DIR_VG_RCNN'] = '/users/zhanwenchen/datasets'
# from maskrcnn_benchmark.data.graft_augmenter import GraftAugmenterDataset

In [3]:
# Set args
# PRETRAINED
from maskrcnn_benchmark.config import cfg

MODEL_NAME = '44663493_vctree_baseline_predcls_4GPU_riv_1_copied'
CONFIG_FILE = '/users/zhanwenchen/gsc/checkpoints/44663493_vctree_baseline_predcls_4GPU_riv_1_copied/config.yml'
PROJECT_DIR = '/users/zhanwenchen/gsc'
SEED=1234
BATCH_SIZE=1

cfg.merge_from_file(CONFIG_FILE)
cfg.SOLVER.IMS_PER_BATCH = BATCH_SIZE
cfg.DATALOADER.NUM_WORKERS = 8
cfg.GLOVE_DIR = f'{PROJECT_DIR}/datasets/vg/'
cfg.MODEL.PRETRAINED_DETECTOR_CKPT = f'{PROJECT_DIR}/checkpoints/pretrained_faster_rcnn/model_final.pth'
cfg.OUTPUT_DIR = f'{PROJECT_DIR}/checkpoints/{MODEL_NAME}'
cfg.PATHS_DATA = f'{PROJECT_DIR}/maskrcnn_benchmark/data/datasets'
cfg.OUTPUT_DIR = '/users/zhanwenchen/gsc/checkpoints/44663493_vctree_baseline_predcls_4GPU_riv_1_copied'
cfg.MODEL.WEIGHT = f'{PROJECT_DIR}/checkpoints/{MODEL_NAME}/model_0014000.pth'
cfg.PATHS_CATALOG = '/users/zhanwenchen/gsc/maskrcnn_benchmark/config/paths_catalog.py'

cfg.freeze()

In [4]:
# graft_augmenter_dataset = GraftAugmenterDataset(cfg)

In [5]:
# import logging
# from torch.utils.data import DataLoader
# from torch.utils.data.sampler import RandomSampler, SequentialSampler, BatchSampler
# from maskrcnn_benchmark.utils.comm import get_world_size
# from maskrcnn_benchmark.data.build import make_batch_data_sampler, make_data_sampler
# from maskrcnn_benchmark.data import samplers
# # from maskrcnn_benchmark.data.build import make_batch_data_sampler, make_data_sampler, _compute_aspect_ratios, _quantize
# from maskrcnn_benchmark.data.transforms import build_transforms
# from maskrcnn_benchmark.data.samplers import DistributedSampler
# from maskrcnn_benchmark.data.collate_batch import BatchCollator, BBoxAugCollator


# # def make_batch_data_sampler(
# #     dataset, sampler, aspect_grouping, images_per_batch, num_iters=None, start_iter=0
# # ):
# #     if aspect_grouping:
# #         if not isinstance(aspect_grouping, (list, tuple)):
# #             aspect_grouping = [aspect_grouping]
# #         aspect_ratios = _compute_aspect_ratios(dataset.dataset)
# #         group_ids = _quantize(aspect_ratios, aspect_grouping)
# #         batch_sampler = samplers.GroupedBatchSampler(
# #             sampler, group_ids, images_per_batch, drop_uneven=False
# #         )
# #     else:
# #         batch_sampler = BatchSampler(
# #             sampler, images_per_batch, drop_last=False
# #         )
# #     if num_iters is not None:
# #         batch_sampler = samplers.IterationBasedBatchSampler(
# #             batch_sampler, num_iters, start_iter
# #         )
# #     return batch_sampler


# def make_data_loader_graft(cfg, mode='train', is_distributed=False, start_iter=0):
#     assert mode in {'train', 'val', 'test'}
#     logger = logging.getLogger(__name__)
#     num_gpus = get_world_size()
#     is_train = mode == 'train'
#     if is_train:
#         images_per_batch = cfg.SOLVER.IMS_PER_BATCH
#         assert (
#             images_per_batch % num_gpus == 0
#         ), "SOLVER.IMS_PER_BATCH ({}) must be divisible by the number of GPUs ({}) used.".format(
#             images_per_batch, num_gpus)
#         images_per_gpu = images_per_batch // num_gpus
#         logger.warning(
#             "Images Per GPU: "+str(images_per_gpu)
#             )
#         shuffle = True
#         num_iters = cfg.SOLVER.MAX_ITER
#     else:
#         images_per_batch = cfg.TEST.IMS_PER_BATCH
#         assert (
#             images_per_batch % num_gpus == 0
#         ), "TEST.IMS_PER_BATCH ({}) must be divisible by the number of GPUs ({}) used.".format(
#             images_per_batch, num_gpus)
#         images_per_gpu = images_per_batch // num_gpus
#         shuffle = False
#         print(f'is_distributed={is_distributed}, shuffle={shuffle}')
#         num_iters = None
#         start_iter = 0

#     if images_per_gpu > 1:
#         logger.warning(
#             "When using more than one image per GPU you may encounter "
#             "an out-of-memory (OOM) error if your GPU does not have "
#             "sufficient memory. If this happens, you can reduce "
#             "SOLVER.IMS_PER_BATCH (for training) or "
#             "TEST.IMS_PER_BATCH (for inference). For training, you must "
#             "also adjust the learning rate and schedule length according "
#             "to the linear scaling rule. See for example: "
#             "https://github.com/facebookresearch/Detectron/blob/master/configs/getting_started/tutorial_1gpu_e2e_faster_rcnn_R-50-FPN.yaml#L14"
#         )

#     # group images which have similar aspect ratio. In this case, we only
#     # group in two cases: those with width / height > 1, and the other way around,
#     # but the code supports more general grouping strategy
#     aspect_grouping = [1] if cfg.DATALOADER.ASPECT_RATIO_GROUPING else []

#     dataset_list = cfg.DATASETS.TRAIN
    
#     dataset_aug = GraftAugmenterDataset(cfg)
#     dataset_train_combined = ([dataset.dataset, dataset])
# #     dataloader_train = DataLoader(dataset=train_dev_sets, ...)

#     # If bbox aug is enabled in testing, simply set transforms to None and we will apply transforms later
#     transforms = None if not is_train and cfg.TEST.BBOX_AUG.ENABLED else build_transforms(cfg, is_train)
#     sampler = make_data_sampler(dataset, shuffle, is_distributed)
#     batch_sampler = make_batch_data_sampler(
#         dataset, sampler, aspect_grouping, images_per_gpu, num_iters, start_iter
#     )
#     collator = BBoxAugCollator() if not is_train and cfg.TEST.BBOX_AUG.ENABLED else \
#         BatchCollator(cfg.DATALOADER.SIZE_DIVISIBILITY)
#     data_loader = DataLoader(
#         dataset,
#         num_workers=cfg.DATALOADER.NUM_WORKERS,
#         batch_sampler=batch_sampler,
#         collate_fn=collator,
#         pin_memory=True,
#     )

#     return data_loader

In [6]:
# dataloader_graft = make_data_loader_graft(cfg, mode='train', is_distributed=False, start_iter=0)

In [7]:
# len(dataloader_graft.dataset.dataset) # 57723

In [8]:
# len(dataloader_graft.dataset) # 14520 # total: 72243

In [9]:
# for iteration, (images, targets, _) in enumerate(dataloader_graft):
#     print(f'i={iteration}')
#     import pdb; pdb.set_trace()

In [10]:
cfg.SOLVER.AUGMENTATION.USE_GRAFT

True

In [15]:
from maskrcnn_benchmark.data import make_data_loader, get_dataset_statistics


train_data_loader = make_data_loader(
        cfg,
        mode='train',
        is_distributed=False,
        start_iter=0,
        aug=True,
    )

Images Per GPU: 1
Images Per GPU: 1


split:  train
root_classes_count:  {}
mean root class number:  0.0
sum root class number:  0
leaf_classes_count:  {}
mean leaf class number:  0.0
sum leaf class number:  0
all_classes_count:  {}
mean all class number:  0.0
sum all class number:  0
number images:  57723
split:  train
root_classes_count:  {}
mean root class number:  0.0
sum root class number:  0
leaf_classes_count:  {}
mean leaf class number:  0.0
sum leaf class number:  0
all_classes_count:  {}
mean all class number:  0.0
sum all class number:  0
number images:  57723
GraftAugmenterDataset.__init__: started running get_dataset_statistics
dataset_name=VG_stanford_filtered_with_attribute_train
split:  train
root_classes_count:  {}
mean root class number:  0.0
sum root class number:  0
leaf_classes_count:  {}
mean leaf class number:  0.0
sum leaf class number:  0
all_classes_count:  {}
mean all class number:  0.0
sum all class number:  0
number images:  57723
get visual genome statistics!!!!!!!!!!!!!!!!!!


100%|███████████████████████████████████| 57723/57723 [00:09<00:00, 5846.29it/s]


GraftAugmenterDataset.__init__: finished running get_dataset_statistics
GraftAugmenterDataset.__init__: started converting stats to df


Dataset [ConcatDataset] has no categories attribute, labels.json file won't be created


GraftAugmenterDataset.__init__: finished converting stats to df
GraftAugmenterDataset.__init__: started converting stats to df
GraftAugmenterDataset.__init__: started querying for least frequent relations
GraftAugmenterDataset.__init__: finished querying for least frequent relations


In [14]:
len(train_data_loader.dataset)

72243

In [ ]:
for iteration, (images, targets, _) in enumerate(train_data_loader):
    print(f'i={iteration}')
    import pdb; pdb.set_trace()

In [ ]:
# Image tensors vs targets mismatch. Does that also happen with the original dataloader? Yes.
# images.tensors.size() # torch.Size([1, 3, 928, 608])
# targets[0].size # (600, 900)

In [ ]:
from torch.utils.data import ConcatDataset

dataset_train_combined = ([train_set, dev_set])
dataloader_train = DataLoader(dataset=train_dev_sets, ...)

In [ ]:
def make_data_loader(cfg, mode='train', is_distributed=False, start_iter=0):
    assert mode in {'train', 'val', 'test'}
    logger = logging.getLogger(__name__)
    num_gpus = get_world_size()
    is_train = mode == 'train'
    if is_train:
        images_per_batch = cfg.SOLVER.IMS_PER_BATCH
        assert (
            images_per_batch % num_gpus == 0
        ), "SOLVER.IMS_PER_BATCH ({}) must be divisible by the number of GPUs ({}) used.".format(
            images_per_batch, num_gpus)
        images_per_gpu = images_per_batch // num_gpus
        logger.warning(
            "Images Per GPU: "+str(images_per_gpu)
            )
        shuffle = True
        num_iters = cfg.SOLVER.MAX_ITER
    else:
        images_per_batch = cfg.TEST.IMS_PER_BATCH
        assert (
            images_per_batch % num_gpus == 0
        ), "TEST.IMS_PER_BATCH ({}) must be divisible by the number of GPUs ({}) used.".format(
            images_per_batch, num_gpus)
        images_per_gpu = images_per_batch // num_gpus
        shuffle = False
        print(f'is_distributed={is_distributed}, shuffle={shuffle}')
        num_iters = None
        start_iter = 0

    if images_per_gpu > 1:
        logger.warning(
            "When using more than one image per GPU you may encounter "
            "an out-of-memory (OOM) error if your GPU does not have "
            "sufficient memory. If this happens, you can reduce "
            "SOLVER.IMS_PER_BATCH (for training) or "
            "TEST.IMS_PER_BATCH (for inference). For training, you must "
            "also adjust the learning rate and schedule length according "
            "to the linear scaling rule. See for example: "
            "https://github.com/facebookresearch/Detectron/blob/master/configs/getting_started/tutorial_1gpu_e2e_faster_rcnn_R-50-FPN.yaml#L14"
        )

    # group images which have similar aspect ratio. In this case, we only
    # group in two cases: those with width / height > 1, and the other way around,
    # but the code supports more general grouping strategy
    aspect_grouping = [1] if cfg.DATALOADER.ASPECT_RATIO_GROUPING else []

    paths_catalog = import_file(
        "maskrcnn_benchmark.config.paths_catalog", cfg.PATHS_CATALOG, True
    )
    DatasetCatalog = paths_catalog.DatasetCatalog
    if mode == 'train':
        dataset_list = cfg.DATASETS.TRAIN
    elif mode == 'val':
        dataset_list = cfg.DATASETS.VAL
    else:
        dataset_list = cfg.DATASETS.TEST

    # If bbox aug is enabled in testing, simply set transforms to None and we will apply transforms later
    transforms = None if not is_train and cfg.TEST.BBOX_AUG.ENABLED else build_transforms(cfg, is_train)
    datasets = build_dataset(cfg, dataset_list, transforms, DatasetCatalog, is_train)

    if is_train:
        # save category_id to label name mapping
        save_labels(datasets, cfg.OUTPUT_DIR)

    data_loaders = []
    for dataset in datasets:
        # print('============')
        # print(len(dataset))
        # print(images_per_gpu)
        # print('============')
        sampler = make_data_sampler(dataset, shuffle, is_distributed)
        batch_sampler = make_batch_data_sampler(
            dataset, sampler, aspect_grouping, images_per_gpu, num_iters, start_iter
        )
        collator = BBoxAugCollator() if not is_train and cfg.TEST.BBOX_AUG.ENABLED else \
            BatchCollator(cfg.DATALOADER.SIZE_DIVISIBILITY)
        num_workers = cfg.DATALOADER.NUM_WORKERS
        data_loader = DataLoader(
            dataset,
            num_workers=num_workers,
            batch_sampler=batch_sampler,
            collate_fn=collator,
            pin_memory=True,
        )
        data_loaders.append(data_loader)
    if is_train:
        # during training, a single (possibly concatenated) data_loader is returned
        assert len(data_loaders) == 1
        return data_loaders[0]
    return data_loaders